In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import os

In [4]:
from postgress_config import PostgressDB

In [5]:
from excel_maker import Support

In [6]:
sup = Support()

In [7]:
postDB = PostgressDB()

In [8]:
df_dp_ip_kh = pd.read_excel(r"resources\ip_payout.xlsx")

In [9]:
df_dp_ip_kh = sup.add_new_column(
            df_dp_ip_kh, "SOURCE OF DATA", "IP Admitting Doc wise rev rep"
        )

In [10]:
 df_dp_op_kh = pd.read_excel(r"resources\op_payout.xlsx")

In [11]:
df_dp_op_kh = sup.add_new_column(
            df_dp_op_kh, "SOURCE OF DATA", "Op Rev share for Ext Doc Rep"
        )

In [12]:
main_dataframe = sup.concat_dataframes(df_dp_ip_kh, df_dp_op_kh)

In [13]:
df_dp_ip_kh = sup.delete_dataframe(df_dp_ip_kh)

In [14]:
df_dp_op_kh = sup.delete_dataframe(df_dp_op_kh)

In [15]:
doctors_list_data, doctors_list_column = postDB.applicable_doctors()

In [16]:
doctors_list_df = pd.DataFrame(
            data=doctors_list_data, columns=list(doctors_list_column)
        )

In [17]:
main_dataframe = main_dataframe[
            main_dataframe["DOCTOR_NAME"]
            .apply(str.upper)
            .isin(doctors_list_df["doctors_name"].apply(str.upper))
        ]

In [18]:
df_dp_rh = pd.read_excel(r"viren_ref\RH_Data.xlsx")

In [19]:
df_dp_rh = sup.add_new_column(
            df_dp_rh, "SOURCE OF DATA", "RH DATA IP Admitting Doc wise rev rep")

In [20]:
df_dp_rh = sup.rename_columns(
            df_dp_rh,
            {
                "BILLING_CLASS": "PATIENT_TYPE",
                "GROSS_AMT": "GROSS_AMOUNT",
                "NET_AMT": "NET_AMOUNT",
            },
        )

In [21]:
df_dp_rh = sup.delete_columns(df_dp_rh, ["SERVICE_GROUP"])

In [22]:
main_dataframe = sup.concat_dataframes(main_dataframe, df_dp_rh)

In [23]:
df_dp_rh = sup.delete_dataframe(df_dp_rh)

In [24]:
doctors_list_df = sup.rename_columns(
            doctors_list_df,
            {"doctors_name": "DOCTOR_NAME", "doctors_group": "DOCTORS_GROUP"},
        )

In [25]:
doctors_list_df = doctors_list_df[["DOCTOR_NAME", "DOCTORS_GROUP"]]

In [26]:
main_dataframe = pd.merge(
            main_dataframe,
            doctors_list_df[["DOCTOR_NAME", "DOCTORS_GROUP"]],
            on="DOCTOR_NAME",
            how="left",
        )

In [27]:
doctors_list_df = sup.delete_dataframe(doctors_list_df)

In [28]:
main_dataframe["REFERENCE_GROUP"] = main_dataframe.apply(lambda row: sup.CINP_OR_CNOP_ANE(row["SERVICE_CODE"]), axis=1)

In [29]:
main_dataframe

,SERVICE_DATE,PATIENT_ID,EPISODE_ID,PATIENT_NAME,SERVICE_CODE,SERVICE_DESC,PATIENT_TYPE,GROSS_AMOUNT,NET_AMOUNT,DOCTOR_SHARE,DOCTOR_NAME,PHYSICIAN_ID,SOURCE OF DATA,DOCTORS_GROUP,REFERENCE_GROUP
0,02/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNOP000010,ER Consultation,Emergency,1500.0,1500.0,1500.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,OP Consultation
1,03/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNIP000046,Doctor Routine Visit - Surgical Oncology,General,1000.0,945.0,945.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,IP Consultation
2,04/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNIP000046,Doctor Routine Visit - Surgical Oncology,General,1000.0,945.0,945.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,IP Consultation
3,05/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNIP000046,Doctor Routine Visit - Surgical Oncology,General,1000.0,945.0,945.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,IP Consultation
4,06/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNIP000046,Doctor Routine Visit - Surgical Oncology,General,1000.0,945.0,945.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,IP Consultation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48103,2022-08-02 00:00:00,RH1000038806,10086196,Master Nachiket D Veer,CNOP000007,Primary Consultation (Premium),Outpatient,1800.0,1800.0,1440.0,DR.VIKAS BASA,51005661,RH DATA IP Admitting Doc wise rev rep,D,OP Consultation
48104,2022-08-16 00:00:00,RH1000039488,10088072,Ms Snehi Harshad Prajapati,CNOP000007,Primary Consultation (Premium),Outpatient,1800.0,1800.0,1440.0,DR.VIKAS BASA,51005661,RH DATA IP Admitting Doc wise rev rep,D,OP Consultation
48105,2022-08-16 00:00:00,RH1000039496,10088089,Master Kiyansh Nagen Biswal,CNOP000007,Primary Consultation (Premium),Outpatient,1800.0,1800.0,1440.0,DR.VIKAS BASA,51005661,RH DATA IP Admitting Doc wise rev rep,D,OP Consultation
48106,2022-08-16 00:00:00,RH1000039503,10088103,Ms Mishika Santosh Nair,CNOP000007,Primary Consultation (Premium),Outpatient,1800.0,1800.0,1440.0,DR.VIKAS BASA,51005661,RH DATA IP Admitting Doc wise rev rep,D,OP Consultation


In [30]:
transplant_dataframe = pd.read_excel(r'viren_ref\transplant.xlsx')

In [31]:
transplant_dataframe =transplant_dataframe.dropna()

In [32]:
transp_df_uhid= list(transplant_dataframe['UHID']) + list(transplant_dataframe['UHID.1'])

In [33]:
postDB = PostgressDB()

In [34]:
transplant_doctor_data, column_name = postDB.transplant_doctors('HPB and Liver Transplant')

In [35]:
transp_df = pd.DataFrame(
            data=transplant_doctor_data,
            columns=list(column_name),
        )

In [36]:
main_dataframe = sup.transplant_working(main_dataframe, transp_df,transp_df_uhid)

D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["DOCTOR_NAME"] = doc
D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["DOCTOR_SHARE"] = (
D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [37]:
postDB = PostgressDB()

In [38]:
transplant_doctor_data, column_name = postDB.transplant_doctors("UROLOGY")

In [39]:
transp_df = pd.DataFrame(
            data=transplant_doctor_data,
            columns=list(column_name),
        )

In [40]:
main_dataframe = sup.transplant_working(
            main_dataframe, transp_df, transp_df_uhid
        )

D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["DOCTOR_NAME"] = doc
D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["DOCTOR_SHARE"] = (


In [41]:
transp_df = sup.delete_dataframe(transp_df)

In [42]:
transplant_dataframe = sup.delete_dataframe(transplant_dataframe)

In [43]:
main_dataframe = sup.srs_distribution(main_dataframe)

In [44]:
sup.excel_generator(excel_data=main_dataframe, page_name="main_dataframe")

'D:\\Python Projects\\doctors_payout\\Scripts\\doctors_payout/main_dataframe-24-Sep-2022-15-55-33-490509.xlsx'